In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv("https://raw.githubusercontent.com/mwitiderrick/kerasDO/master/HR_comma_sep.csv")

In [2]:
df.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,department,salary
0,0.38,0.53,2,157,3,0,1,0,sales,low
1,0.80,0.86,5,262,6,0,1,0,sales,medium
2,0.11,0.88,7,272,4,0,1,0,sales,medium
3,0.72,0.87,5,223,5,0,1,0,sales,low
4,0.37,0.52,2,159,3,0,1,0,sales,low


In [3]:
feats = ['department','salary']
df_final = pd.get_dummies(df,columns=feats,drop_first=True)

In [4]:
from sklearn.model_selection import train_test_split

In [5]:
X = df_final.drop(['left'],axis=1).values
y = df_final['left'].values

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [7]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [8]:
import keras
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


In [9]:
classifier = Sequential()

In [10]:
classifier.add(Dense(9, kernel_initializer = "uniform",activation = "relu", input_dim=18))

In [11]:
classifier.add(Dense(1, kernel_initializer = "uniform",activation = "sigmoid"))

In [12]:
classifier.compile(optimizer= "adam",loss = "binary_crossentropy",metrics = ["accuracy"])

In [13]:
classifier.fit(X_train, y_train, batch_size = 10, epochs = 1)

Epoch 1/1
10499/10499 [==============================] - 1s 120us/step - loss: 0.4242 - acc: 0.8174


In [14]:
y_pred = classifier.predict(X_test)

In [15]:
y_pred = (y_pred > 0.5)

In [16]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
cm

array([[3262,  222],
       [ 196,  820]])

In [17]:
new_pred = classifier.predict(sc.transform(np.array([[0.26,0.7 ,3., 238., 6., 0.,0.,0.,0., 0.,0.,0.,0.,0.,1.,0., 0.,1.]])))

In [18]:
new_pred = (new_pred > 0.5)
new_pred

array([[False]], dtype=bool)

In [43]:
new_pred = (new_pred > 0.6)
new_pred

array([[False]], dtype=bool)

In [19]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score

In [20]:
def make_classifier():
    classifier = Sequential()
    classifier.add(Dense(9, kernel_initializer = "uniform", activation = "relu", input_dim=18))
    classifier.add(Dense(1, kernel_initializer = "uniform", activation = "sigmoid"))
    classifier.compile(optimizer= "adam",loss = "binary_crossentropy",metrics = ["accuracy"])
    return classifier

In [21]:
classifier = KerasClassifier(build_fn = make_classifier, batch_size=10, nb_epoch=1)

In [25]:
accuracies = cross_val_score(estimator = classifier,X = X_train,y = y_train,cv = 2,n_jobs = -1)

In [26]:
mean = accuracies.mean()
mean

0.79692891811568356

In [27]:
variance = accuracies.var()
variance

0.0018715620131483841

In [28]:
from keras.layers import Dropout

classifier = Sequential()
classifier.add(Dense(9, kernel_initializer = "uniform", activation = "relu", input_dim=18))
classifier.add(Dropout(rate = 0.1))
classifier.add(Dense(1, kernel_initializer = "uniform", activation = "sigmoid"))
classifier.compile(optimizer= "adam",loss = "binary_crossentropy",metrics = ["accuracy"])

In [29]:
from sklearn.model_selection import GridSearchCV
def make_classifier(optimizer):
    classifier = Sequential()
    classifier.add(Dense(9, kernel_initializer = "uniform", activation = "relu", input_dim=18))
    classifier.add(Dense(1, kernel_initializer = "uniform", activation = "sigmoid"))
    classifier.compile(optimizer= optimizer,loss = "binary_crossentropy",metrics = ["accuracy"])
    return classifier

In [30]:
classifier = KerasClassifier(build_fn = make_classifier)

In [42]:
params = {
    'batch_size':[20,35],
    'nb_epoch':[5,10],
    'optimizer':['adam','rmsprop']
}

In [37]:
grid_search = GridSearchCV(estimator=classifier,
                           param_grid=params,
                           scoring="accuracy",
                           cv=10)

In [38]:
grid_search = grid_search.fit(X_train,y_train)

Epoch 1/1
5249/5249 [==============================] - 1s 148us/step - loss: 0.5681 - acc: 0.7535
Epoch 1/1
5250/5250 [==============================] - 1s 148us/step - loss: 0.5789 - acc: 0.7583
Epoch 1/1
5249/5249 [==============================] - 1s 148us/step - loss: 0.6003 - acc: 0.7512
Epoch 1/1
5250/5250 [==============================] - 1s 174us/step - loss: 0.6069 - acc: 0.7541
Epoch 1/1
5249/5249 [==============================] - 1s 164us/step - loss: 0.5777 - acc: 0.7529
Epoch 1/1
5250/5250 [==============================] - 1s 174us/step - loss: 0.5866 - acc: 0.7562
Epoch 1/1
5249/5249 [==============================] - 1s 167us/step - loss: 0.5981 - acc: 0.7508
Epoch 1/1
5250/5250 [==============================] - 1s 171us/step - loss: 0.5926 - acc: 0.7575
Epoch 1/1
5249/5249 [==============================] - 1s 156us/step - loss: 0.6319 - acc: 0.7531
Epoch 1/1
5250/5250 [==============================] - 1s 165us/step - loss: 0.6432 - acc: 0.7411
Epoch 1/1
5249/5249 

In [39]:
best_param = grid_search.best_params_
best_accuracy = grid_search.best_score_

In [40]:
best_param

{'batch_size': 35, 'nb_epoch': 1, 'optimizer': 'rmsprop'}

In [41]:
best_accuracy

0.7697875988189351